In [1]:
cd documents

C:\Users\Admin\documents


In [20]:
## !pip install googlemaps ##install googlemaps
import googlemaps
import time
import csv

api_key = 'YOUR API KEY'   ### import google map API key
gmaps = googlemaps.Client(key=api_key)

search_category = 'dementia community' ## search key word which decides what category are searched
category_id = '71' ## the category id based on the search key word

params = {
    'query' : [search_category], ### search content
    'location': (-37.8102, 144.9628),  ### default location melbourne central
    'radius': 50000                       ### search radius
}
places = gmaps.places(**params)

csv_list = []
image_max_width = '800'
image_max_height = '800'

places_list = places['results']
for i in range(len(places_list)):
    place_detail = {}
    place_detail['name'] = places_list[i]['name']
    place_detail['address'] = places_list[i]['formatted_address']
    place_detail['lat'] = places_list[i]['geometry']['location']['lat']
    place_detail['lng'] = places_list[i]['geometry']['location']['lng']
    place_detail['rating'] = places_list[i]['rating']
    place_detail['types'] = places_list[i]['types']
    if 'photos' in places_list[i]:
        image_ref = places_list[i]['photos'][0]['photo_reference']
        place_detail['image'] = 'https://maps.googleapis.com/maps/api/place/photo?maxwidth='\
        + image_max_width + '&maxheight=' + image_max_height + '&photoreference=' + image_ref + '&key=' + api_key
    else:
        place_detail['image'] = ''
    csv_list.append(place_detail)
    
### if ther more than 20 record, search next page
try:
    params['page_token'] = places['next_page_token']
    time.sleep(2)
    places_2 = gmaps.places(**params)
except KeyError:
    pass
try:
    for i in place_2['results']:
        csv_list.append(i)
except NameError:
    pass

## column names based on the GeoDirectory sample CSV file
column_name = ['ID','post_title', 'post_author', 'post_category', 'post_tags', 'post_type',\
               'post_status', 'street', 'city', 'region','country', 'latitude', 'longitude',\
               'post_images','video','zip','business_hours','phone','email','website','twitter',\
               'facebook','featured','post_content']

with open('places.csv', 'w') as csvFile:
    writer = csv.writer(csvFile, delimiter=',')
    writer.writerow(column_name)
    for i in csv_list:
        one_place = ['']
        one_place.append(i['name'])
        one_place.append('1')
        one_place.append(category_id)
        one_place.append(','.join(i['types']))
        one_place.append('gd_place')
        one_place.append('publish')
        one_place.append(i['address'])
        one_place.append('Melbourne')
        one_place.append('Victoria')
        one_place.append('Australia')
        one_place.append(i['lat'])
        one_place.append(i['lng'])
        one_place.append(i['image'])
        writer.writerow(one_place)
csvFile.close()